In [1]:
from jeroHelper.setupUtils import append_deepfinder_path, get_tomo_indices, PARENT_PATH
append_deepfinder_path()

tomo_ids, tomo_idx = get_tomo_indices()
print('\n')

from jeroHelper.coordGen import make_xml_objlist_from_crops
from jeroHelper.trainHelper import make_trainer

from deepfinder.training_pylit import TargetBuilder
from deepfinder.dataloader_pylit import DeepFinder_dataset, to_categorical, transpose_to_channels_first
from deepfinder.model_pylit import DeepFinder_model
import deepfinder.utils.objl as ol

import numpy as np
import matplotlib.pyplot as plt
import random

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

Pairs of tomo IDs to indices:
[('tomo02', 0), ('tomo03', 1), ('tomo04', 2), ('tomo10', 3), ('tomo17', 4), ('tomo32', 5), ('tomo38', 6)]




In [2]:
train_tomos = ['tomo02', 'tomo03', 'tomo04', 'tomo17']
concat_train_ids = sorted([s.replace('tomo', '') for s in train_tomos])
concat_train_ids = '-'.join(concat_train_ids)

val_tomos = ['tomo32', 'tomo10']
concat_val_ids = sorted([s.replace('tomo', '') for s in val_tomos])
concat_val_ids = '-'.join(concat_val_ids)


test_tomos = ['tomo38']

crops_coords_str = '309-618_309-618_100-350'

#### This only makes sense if ONE crop of each tomogram is used
path_data = []
path_target = []

# For  baseline
data_template_str = 'data/processed0/nnUnet/cET_cropped/%s_bin4_denoised_0000_%s.mrc'
target_template_str = 'data/processed0/nnUnet/cET_cropped/%s_merged_thr02_lbl_%s.mrc'

for tomo_id, deepFinder_idx in zip(tomo_ids, tomo_idx):

    file_data = PARENT_PATH+data_template_str %(tomo_id, crops_coords_str)
    file_target = PARENT_PATH+target_template_str %(tomo_id, crops_coords_str)
    
    path_data+=[file_data]
    path_target+=[file_target]
    
path_objl_train = '../data/processed0/deepFinder/object_lists/train_tomo%s_%s.xml' %(concat_train_ids, crops_coords_str)
path_objl_valid = '../data/processed0/deepFinder/object_lists/validation_tomo%s_%s.xml' %(concat_val_ids, crops_coords_str)

# Load object lists:
objl_train = ol.read_xml(path_objl_train)
objl_valid = ol.read_xml(path_objl_valid)

random.seed(1)

rsample_train = random.sample(objl_train, 500)
rsample_val = random.sample(objl_valid, 300)

In [3]:
tb_logdir = './logs/2.00_baselineModel/'
# pretrained_model = PARENT_PATH+'models/2.01_proxyLabelModel_ep1__in32_lr0.000100_v0.model'
pretrained_model = None

trainer = make_trainer(dim_in=32, batch_size=32, lr=1e-4, epochs=2, tb_logdir=tb_logdir, model_name='2.00_baselineModel',
                       reconstruction_trainer=False, pretrained_model=pretrained_model)
trainer.launch(path_data, path_target, rsample_train, rsample_val)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | loss_fn | Tversky_loss | 0     
1 | layer1  | Sequential   | 28.6 K
2 | layer2  | Sequential   | 103 K 
3 | layer3  | Sequential   | 558 K 
4 | layer4  | Sequential   | 288 K 
5 | layer5  | Sequential   | 96.9 K
-----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.304     Total estimated model params size (MB)


"Ncl":           2
"loss_fn":       Tversky_loss()
"lr":            0.0001
"pretrain_type": None
"weight_decay":  0.0
Saving model at:  /home/jeronimo/Desktop/Master_mathematics/Thesis/models/2.00_baselineModel_ep2_in32_lr0.000100_v9.model
[tensor(1.0307), tensor(0.9499), tensor(0.5043)] [tensor(1.0041), tensor(0.7415)]
Trying to open file...


In [60]:
save_folder = '/home/jeronimo/Desktop/Master_mathematics/Thesis/models/model_summary'

header = """Name,Version,Training set,# Samples training,Validation set,
# Samples validation,Training epochs,Pretrained type,Pretrained model path,
Loss function,Best epoch validation Loss,
Train Loss (From best validation epoch), Best validation epoch,Comment"""

from pathlib import Path
import numpy as np
import os
import io

def write_modelSummary_row(modelSummary_folder, row_data):
    
    if not Path(modelSummary_folder).is_dir():
        print("Creating modelSummary folder...")
        os.mkdir(modelSummary_folder)
        
    print("Trying to open file...")
    try: 
        file = open(modelSummary_folder+"/model_summary.csv", "r+")
        # check header
        first_line = next(file).replace('\n', '')
        assert first_line==header.replace('\n', '')

        file.writelines('\n'+row_data)

    except IOError:
        print("No file found, writing header and row_data...")
        file = open(modelSummary_folder+"/model_summary.csv", "a+")
        file.writelines([header.replace('\n', ''), "\n"+row_data])

    finally:
        file.close()
    
    return

def write_modelSummary(trainer, modelSummary_folder, path_data, path_target, rsample_train, rsample_val):
    p = Path(trainer.tensorboard_logdir+'default/')
    p = [f.as_posix() for f in list(p.glob('*'))]
    last_version = sorted([int(f.split('version_')[-1]) for f in p])[-1]
    last_version = 'v'+ str(last_version)
    
    print(trainer.model.val_loss_per_epoch, trainer.model.train_loss_per_epoch)
    
    best_val_loss_epoch = np.min(trainer.model.val_loss_per_epoch)
    best_val_loss_epoch_idx = np.argmin(trainer.model.val_loss_per_epoch)
    # pytorch lightning includes epoch zero (no training) in the validation
    associated_train_loss_epoch = trainer.model.train_loss_per_epoch[best_val_loss_epoch_idx-1]
    
    tomograms = [f.split('/')[-1] for f in path_data]
    tomograms = np.array([f.split('_')[0] for f in tomograms])

    _tomograms = [f.split('/')[-1] for f in path_target]
    _tomograms = np.array([f.split('_')[0] for f in _tomograms])

    assert all(tomograms == _tomograms)

    train_set = np.unique([x['tomo_idx'] for x in rsample_train])
    train_set = ' '.join(tomograms[train_set])

    validation_set = np.unique([x['tomo_idx'] for x in rsample_val])
    validation_set = ' '.join(tomograms[validation_set])
    
    
    row_data = [trainer.model_name, last_version, train_set, len(rsample_train), validation_set, len(rsample_val), trainer.epochs,
                trainer.pretrain_type, trainer.pretrained_model, trainer.loss_fn, best_val_loss_epoch, float(associated_train_loss_epoch),
                best_val_loss_epoch_idx]
    
    row_data =','.join([str(x) for x in row_data])
    write_modelSummary_row(modelSummary_folder, row_data)
    
    return

In [61]:
write_modelSummary(trainer, save_folder, path_data, path_target, rsample_train, rsample_val)

[tensor(1.0307), tensor(0.9499), tensor(0.5043)] [tensor(1.0041), tensor(0.7415)]
Trying to open file...


In [57]:


validation_set

'tomo10, tomo32'

True

In [27]:
import re

In [31]:
p = re.compile(r'^tomo\d+')
p.findall('tomo12 drummers drumming, 11 pipers piping, 10 lords a-leaping')


['thing tomo12']